In [3]:
DO_TOKENIZATION=True
TRAIN_MODEL=False
# DATA_PREFIX='expt_1'
DATA_PREFIX='tiny_sample'
MODEL_EXPT_NAME="multitok_model_1"

TRAIN_DATA_PATH=f'train-model/{DATA_PREFIX}'
MODEL_PATH=f'train-model/{DATA_PREFIX}/{MODEL_EXPT_NAME}'

In [7]:
!pip install lingua 
!pip install multi-tokenizer
!pip install ipython-unittest


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [10]:
from datasets import load_dataset
from itertools import chain
import tokenizers
    
import sys

from tqdm import tqdm


import os
import os.path
import random
import re
import torch

import torch.nn as nn
from torch.nn import functional as F
from itertools import islice
from typing import List

%load_ext ipython_unittest

from multi_tokenizer import LanguageDetector
from lingua import DetectionResult, Language as LinguaLanguage 


device= 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)
# assert device == 'cuda', "This notebook is not optimized for CPU"
if device == 'cpu' and TRAIN_MODEL:
    print("Warning: Training on CPU will be slow")
    assert False

config = {
    "learning_rate": 1e-3,
    "eval_interval": 300,
    "max_iters": 60000, 
    "H": 32, # per head dimension size
    "B": 64, # batch size
    "T": 256, # Sequence length
    "C": 256, # model size
    "feedforward_factor": 3,
    "n_heads": 8,
    "dropout": 0.0,
    "l2_penalty": 0.0,
    "n_layers": 12,
    "vocab_size": 2**13,
    # "git_hash": os.popen("git rev-parse HEAD").read().strip()
}

for k,v in config.items():
    locals()[k] = v

In [13]:
%%unittest_testcase 
def batch_iterator(iterable, batch_size):
    iterator = iter(iterable)
    return iter(lambda: list(islice(iterator, batch_size)), [])

def test_sample_batch_iterator(self):
    data = list(range(5))
    l = batch_iterator(data, 2)
    self.assertEqual(next(l), [0, 1])
    self.assertEqual(next(l), [2, 3])
    self.assertEqual(next(l), [4])
    with self.assertRaises(StopIteration):
        next(l)


Success

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [ ]:
'''
For The input train txt file, below will be the output format given by running this hacky lang detect code:

Input text file:

[PROMPT] español [USER] Un niñ [END]
[PROMPT] español [USER] Hola [END]
[PROMPT] english [USER] Hi [END]
[PROMPT] español [USER] a child [END]
[PROMPT] translate [USER] It was night
Era de noche 
Hi
Hola [END]
[PROMPT] english [USER] It was a night [END]



Output text file with appended lang ids:

[BEGIN-ES] [PROMPT] español [USER] Un niñ [END]
[PROMPT] español [USER] Hola [END]
[BEGIN-EN] [PROMPT] english [USER] Hi [END]
[BEGIN-ES] [PROMPT] español [USER] a child [END]
[BEGIN-EN] [PROMPT] translate [USER] It was night
[BEGIN-ES] Era de noche
[BEGIN-EN] Hi
[BEGIN-ES] Hola [END]
[BEGIN-EN] [PROMPT] english [USER] It was a night [END]

'''



END_TOKEN = '[END]'
PROMPT_TOKEN = '[PROMPT]'

from typing import List

from lingua import DetectionResult, Language, LanguageDetectorBuilder


class HackyCheatingLangDetector:
    def __init__(self, languages: List[Language]) -> None:
        """Initialize Language Detector."""
        self.languages = languages
        self.detector = LanguageDetectorBuilder.from_languages(*languages).build()

    def _delimited_stories_iterator(self, text):
        start = 0
        last_match_end = text.find(END_TOKEN)
        while last_match_end != -1:
            yield text[start:last_match_end + len(END_TOKEN)]
            start = last_match_end + len(END_TOKEN) #+ 1 # Add one to skip the space
            last_match_end = text.find(END_TOKEN, start)

    def _detect_single_story(self, story_text) -> List[DetectionResult]:
        # story_text = story_text.strip()
        # assert story_text.startswith(PROMPT_TOKEN), f"Unexpected start: {story_text[:20]}"
        task = story_text.split(' ', 2)[1]
        assert task in ['english', 'español', 'translate']
        paragraph_splitter = re.compile('[^\n]+')
        if task == 'english':
            return [DetectionResult(0, len(story_text), 0, LinguaLanguage.ENGLISH)]
        elif task == 'español':
            return [DetectionResult(0, len(story_text), 0, LinguaLanguage.SPANISH)]
        elif task == 'translate':
            paragraph_starts = [m.span()[0] for m in re.finditer(paragraph_splitter, story_text)]
            ret = []
            for paragraph_no, paragraph_start in enumerate(paragraph_starts):
                next_paragraph_start = paragraph_starts[paragraph_no + 1] if paragraph_no + 1 < len(paragraph_starts) else len(story_text)+1
                lang = LinguaLanguage.ENGLISH if paragraph_no % 2 == 0 else LinguaLanguage.SPANISH
                ret.append(DetectionResult(paragraph_start, next_paragraph_start, 0, lang))
            return ret

  
    def split_n_detect(self, text: str, sep: str = " ") -> List[DetectionResult]:
        """Split Text and Detect Language."""

        def merge_results(
            results: List[List[DetectionResult]],
        ) -> List[DetectionResult]:
            """Merge Results. If consecutive words are detected as the same language, merge them."""
            merged_results: list[DetectionResult] = []
            for result in results:
                if not merged_results:
                    merged_results.extend(result)
                else:
                    for detection in result:
                        last_result = merged_results[-1]
                        if detection.language == last_result.language:
                            merged_results[-1] = DetectionResult(
                                language=last_result.language,
                                start_index=last_result.start_index,
                                end_index=last_result.end_index
                                + detection.end_index
                                - detection.start_index,
                                word_count=last_result.word_count
                                + detection.word_count,
                            )
                        else:
                            merged_results.append(
                                DetectionResult(
                                    language=detection.language,
                                    start_index=last_result.end_index,
                                    end_index=last_result.end_index
                                    + detection.end_index
                                    - detection.start_index,
                                    word_count=detection.word_count,
                                )
                            )
            return merged_results


        results = []
        # Detect the language of each part
        for story_text in self._delimited_stories_iterator(text):
            results.append(self._detect_single_story(story_text))
        
        # Merge consecutive results with the same language
        merged_results = merge_results(results)
        return merged_results


supported_languages = [LinguaLanguage.ENGLISH, LinguaLanguage.SPANISH]
lang_detector = HackyCheatingLangDetector(supported_languages)
monolingual_token_file_names = {l: f'{MODEL_PATH}/train_tokenizer_{l}.txt' for l in supported_languages}
lang_to_token_map = {LinguaLanguage.ENGLISH: '[BEGIN-EN]', LinguaLanguage.SPANISH: '[BEGIN-ES]'}


os.makedirs(MODEL_PATH, exist_ok=True)
outputs_split_by_lang = {}
for l in supported_languages:        
    outputs_split_by_lang[l] = open(monolingual_token_file_names[l], 'w')
segmented_output = open(f'{MODEL_PATH}/train_segmented.txt', 'w')
    
# instead of batch iterator, split first by END_TOKEN, and then split_n_detect on fragments containing
# multiple END_TOKENs, split by END_TOKEN
for buffered_lines in tqdm(batch_iterator(open(f'{TRAIN_DATA_PATH}/train_tiny.txt'), 10000)):        
    joined_line = ''.join(buffered_lines)
    for detection in lang_detector.split_n_detect(joined_line, END_TOKEN): 
        # print(detection) 
        monolingual_fragment = joined_line[detection.start_index: detection.end_index].strip()
        # print(monolingual_fragment)
        print(f"{lang_to_token_map[detection.language]} {monolingual_fragment}") 
        # outputs_split_by_lang[detection.language].write(monolingual_fragment + '\n')
        segmented_output.write(f"{lang_to_token_map[detection.language]} {monolingual_fragment}\n")

In [ ]:
END_TOKEN = '[END]'
PROMPT_TOKEN = '[PROMPT]'

def delimited_stories_iterator(text):
    start = 0
    last_match_end = text.find(END_TOKEN)
    while last_match_end != -1:
        yield text[start:last_match_end + len(END_TOKEN)]
        start = last_match_end + len(END_TOKEN)
        last_match_end = text.find(END_TOKEN, start)
delimited_stories = list(delimited_stories_iterator
                         ('foo bar [END] baz [END] quux [END]'))
assert delimited_stories == ['foo bar [END]', ' baz [END]', ' quux [END]']


paragraph_splitter = re.compile('[^\n]+')

class HackyCheatingLangDetector:
    def __init__(self, supported_langs):
        pass

    def _detect_single_story(self, story_text) -> List[DetectionResult]:
        assert story_text.startswith(PROMPT_TOKEN)
        task = story_text.split(' ', 2)[1]
        assert task in ['english', 'español', 'translate']
        if task == 'english':
            return [DetectionResult(0, len(story_text), -1, LinguaLanguage.ENGLISH)]
        elif task == 'español':
            return [DetectionResult(0, len(story_text), -1, LinguaLanguage.SPANISH)]
        elif task == 'translate':
            paragraph_starts = [m.span()[0] for m in re.finditer(paragraph_splitter, story_text)]
            ret = []
            for paragraph_no, paragraph_start in paragraph_starts:
                
                lang = LinguaLanguage.ENGLISH if paragraph_no % 2 == 0 else LinguaLanguage.SPANISH
                ret.append(DetectionResult(lang, ))
        
            

    def split_n_detect(self, text, sep) -> List[DetectionResult]:
        assert sep == '[END]'
        for story_start_index, story_text in delimited_stories_iterator(text):
            # _detect_single_story + adjust (add indexes) to be global to text rather than local to story_text
                


In [8]:
import re
splits = paragraph_splitter.split('foo\nbar\n\nbaz\nquux\n\n\netc\n')
print(splits)

['', '\n', '\n\n', '\n', '\n\n\n', '\n']


In [14]:
#[m for m in paragraph_splitter.finditer('\n+', "foo\nbar\n\nbaz\netc")]
[m.span()[0] for m in paragraph_splitter.finditer('foo\nbar\n\nbaz\nquux\n\n\netc\n')]

[0, 4, 9, 13, 20]

In [75]:
special_tokens = ['[PROMPT]', '[USER]', '[END]', '[PAD]', '[BEGIN-EN]', '[BEGIN-ES]']
ENGLISH, SPANISH = 0, 1
lang_to_token_map = {LinguaLanguage.ENGLISH: '[BEGIN-EN]', LinguaLanguage.SPANISH: '[BEGIN-ES]'}
lingua_lang_to_dense_lang_id = {LinguaLanguage.ENGLISH: 0, LinguaLanguage.SPANISH: 1}
supported_languages = [LinguaLanguage.ENGLISH, LinguaLanguage.SPANISH]
monolingual_token_file_names = {l: f'{MODEL_PATH}/train_tokenizer_{l}.txt' for l in supported_languages}

os.makedirs(MODEL_PATH, exist_ok=True)

if DO_TOKENIZATION:
    lang_detector = LanguageDetector(supported_languages)

    os.makedirs(MODEL_PATH, exist_ok=True)
    outputs_split_by_lang = {}
    for l in supported_languages:        
        outputs_split_by_lang[l] = open(monolingual_token_file_names[l], 'w')
    segmented_output = open(f'{MODEL_PATH}/train_segmented.txt', 'w')
        
    # instead of batch iterator, split first by END_TOKEN, and then split_n_detect on fragments containing
    # multiple END_TOKENs, split by END_TOKEN
    for buffered_lines in tqdm(batch_iterator(open(f'{TRAIN_DATA_PATH}/train_tiny.txt'), 10000)):        
        joined_line = ''.join(buffered_lines)
        for detection in lang_detector.split_n_detect(joined_line, END_TOKEN):  
            monolingual_fragment = joined_line[detection.start_index: detection.end_index] 
            # outputs_split_by_lang[detection.language].write(monolingual_fragment + '\n')
            segmented_output.write(f"{lang_to_token_map[detection.language]} {monolingual_fragment}'\n")
    
    for l in supported_languages:
        outputs_split_by_lang[l].close()
    segmented_output.close()

        

1it [00:00, 165.45it/s]


In [81]:
sample = open(f'{TRAIN_DATA_PATH}/train_tiny.txt').read()
for detection in lang_detector.split_n_detect(sample, '[END]'):
    print(str(detection.language) + ' '+ sample[detection.start_index: detection.end_index])

Language.SPANISH [PROMPT] español [USER] Un niño pequeño jugó en el parque. Él vio un perro grande. "Hola", dijo el niño. El perro movió la cola. "¡Qué suerte!", dijo el niño. "Puedo jugar contigo".

El niño tiró la pelota. El perro la persiguió. ¡Fue divertido! El niño corrió. El perro corrió. ¡Ellos eran amigos! 

El niño vio un carro. ¡Era rojo! "Mira", dijo el niño al perro. "¡Ese carro es rojo!" El perro olfateó. El niño rió. "¡Qué suerte!" dijo el niño. "Puedo jugar más tiempo".

De repente, el carro se detuvo. Una señora salió. "¡Ese es mi perro!", dijo la señora. "¡Podías jugar con él, pero ahora es hora de irnos!". 

El niño se puso triste. Él quería jugar más. La señora le dio una galleta. El niño sonrió. "¡Qué suerte!", dijo el niño. "¡Tengo una galleta!" [END]
[PROMPT] español [USER] Una niña llamada Ana cumple años. ¡Hurra! Ana tiene muchísimas amigas que van a su fiesta. ¡Es divertido! Ana abre sus regalos. ¡Wow! Un carro rojo, una pelota grande y... ¡un diamante! ¡Es bri

In [35]:
tokenizers_by_dense_lang_id = []

if DO_TOKENIZATION:
    for l in supported_languages:
        tokenizer = tokenizers.ByteLevelBPETokenizer()
        tokenizer.train(
            files=[monolingual_token_file_names[l]], 
            vocab_size=vocab_size, 
            min_frequency=2,
            special_tokens=special_tokens
        )
        tokenizer.add_special_tokens(special_tokens)
        tokenizer.save_model(MODEL_PATH, f'tiny-stories-{l}-bpe')
        tokenizers_by_dense_lang_id.append(tokenizer)
else:
    for l in supported_languages:
        tokenizer = tokenizers.ByteLevelBPETokenizer(
            f'{MODEL_PATH}/tiny-stories-{l}-bpe-vocab.json', 
            f'{MODEL_PATH}/tiny-stories-{l}-bpe-merges.txt'
        )
        tokenizer.add_special_tokens(special_tokens)
        tokenizers_by_dense_lang_id.append(tokenizer)


In [25]:
PROMPT_TOK_ID = trained_tokenizers_by_dense_lang_id[0].token_to_id('[PROMPT]')
USER_TOK_ID = trained_tokenizers_by_dense_lang_id[0].token_to_id('[USER]')
END_TOK_ID = trained_tokenizers_by_dense_lang_id[0].token_to_id('[END]')
PAD_TOK_ID = trained_tokenizers_by_dense_lang_id[0].token_to_id('[PAD]')
BEGIN_EN_TOK_ID = trained_tokenizers_by_dense_lang_id[0].token_to_id('[BEGIN-EN]')
BEGIN_ES_TOK_ID = trained_tokenizers_by_dense_lang_id[0].token_to_id('[BEGIN-ES]')

LANG_BEGIN_TOK_IDS = [BEGIN_EN_TOK_ID, BEGIN_ES_TOK_ID]

In [43]:
lang_splitter_re = re.compile(r'\[BEGIN-(EN|ES)\]')
lang_code_to_lang_dense_id = {'EN': ENGLISH, 'ES': SPANISH}

def multitokenize_delimited_story(text):
    assert text.startswith('[BEGIN-EN]') or text.startswith('[BEGIN-ES]')
    assert text.endswith(END_TOKEN)

    # This splits text into sequences of [lang_code_1, text_1, lang_code_2, text_2, ...]
    # where lang_code_i is either 'EN' or 'ES'
    per_lang_splits = [p.strip() for p in lang_splitter_re.split(text) if p.strip()]
    assert len(per_lang_splits) % 2 == 0
    lang_dense_id_per_text = [lang_code_to_lang_dense_id[code] for code in per_lang_splits[::2]]
    monolingual_texts = per_lang_splits[1::2]

    token_ids, lang_ids = [], []
    for lang_dense_id, monolingual_text in zip(lang_dense_id_per_text, monolingual_texts):
        tokenizer = tokenizers_by_dense_lang_id[lang_dense_id]
        
        these_tokens = [LANG_BEGIN_TOK_IDS[lang_dense_id]] + tokenizer.encode(monolingual_text).ids 
        token_ids.extend(these_tokens)
        lang_ids.extend([lang_dense_id] * len(these_tokens))
    
    return torch.tensor(token_ids, dtype=torch.short), torch.tensor(lang_ids, dtype=torch.short)
test_input = (
    """[BEGIN-EN] [PROMPT] translate [USER] One day, a little boy saw a cake. It was very big and had many strawberries.
[BEGIN-ES] Un día, un niño pequeño vio un pastel. Era muy grande y tenía muchas fresas.
[BEGIN-EN] Slightly more text [END]""")
# multitokenize_delimited_story(test_input)


(tensor([   4,    0, 1463,  226,    1,  511,  363,   17,  264,  419,  513,  431,
          264, 1126,   19,  424,  286,  409,  426,  270,  369,  794, 4451,   19,
            5,  351,  443,   17,  288,  324,  431,  427,  288,  640,   19,  470,
          353,  414,  314,  410,  876, 1861,   19,    4,   56, 2770,  413,  712,
          948,  973,  226,    2], dtype=torch.int16),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0], dtype=torch.int16))

In [ ]:
# this is what needs to be replaced 

if DO_TOKENIZATION:
    # stories = chain(tinystories_ds['train']['text'], tinystories_sp_ds['train']['story'])
    stories = tinystories_sp_ds['train']
    
    if not os.path.isdir('tokenized'):
        os.mkdir('tokenized')
    output_buf = []
    num_outputs = 0
    for story in tqdm(stories):
        encoded_en = torch.tensor([START_TOK, BEGIN_EN_TOK] + 
                                tokenizer_en.encode(story['translation']).ids + 
                                [END_TOK], dtype=torch.short)
        encoded_es = torch.tensor([START_TOK, BEGIN_ES_TOK] + 
                                tokenizer_es.encode(story['story']).ids + 
                                [END_TOK], dtype=torch.short)


        en_lang_idxs = torch.zeros_like(encoded_en)
        es_lang_idxs = torch.ones_like(encoded_es)
        
        output_buf.append([encoded_en, en_lang_idxs])
        output_buf.append([encoded_es, es_lang_idxs])

        if len(output_buf) > 500_000:
            torch.save(output_buf, f'tokenized/tokenized-{num_outputs}.pt')
            num_outputs += 1
            output_buf = []
    if output_buf:
        torch.save(output_buf, f'tokenized/tokenized-{num_outputs}.pt')
        num_outputs += 1
        output_buf = []

100%|██████████| 100000/100000 [01:09<00:00, 1447.83it/s]


## Train

In [112]:
def load_sharded_story(shard_no):
    return torch.load(f'tokenized/tokenized-{shard_no}.pt')

In [113]:
# load the tokenized stories in parallel using threads
# this is faster than loading them sequentially
num_shards = len(os.listdir('tokenized'))
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor() as pool:
    stories = list(tqdm(pool.map(load_sharded_story, range(num_shards)), total=num_shards))

100%|██████████| 1/1 [00:12<00:00, 12.61s/it]


In [114]:
all_stories = []
for story in stories:
    all_stories.extend(story)
random.seed(1337)
random.shuffle(all_stories)

In [115]:
print("length of dataset in stories: ", len(all_stories))
print("length of stories in tokens", sum(len(story) for story in all_stories))

length of dataset in stories:  200000
length of stories in tokens 400000


In [116]:
num_stories_to_check = 1_000_000
num_long = sum(len(story[0]) > T for story in all_stories[:num_stories_to_check])
print(
    f"# stories longer than {T} : {num_long} out of {num_stories_to_check}, {num_long/num_stories_to_check:.2%}")

# stories longer than 256 : 10705 out of 1000000, 1.07%


In [117]:
def encode(text, language_id):
    if language_id == 0:
        tokenizer = tokenizer_en
    else:
        tokenizer = tokenizer_es
    return tokenizer.encode(text).ids
def decode(encoded_text, language_id):
    if language_id == 0:
        tokenizer = tokenizer_en
    else:
        tokenizer = tokenizer_es
    return tokenizer.decode(encoded_text)

from tqdm import tqdm



print("Hola encoded in english")
hello_encoded = encode("Hola", 0)
print(hello_encoded)

print("Hola encoded in spani.tqdmsh")
hello_encoded = encode("Hola", 1)
print(hello_encoded)
print(decode(hello_encoded, 0))

vocab_size = tokenizer_en.get_vocab_size()
print("vocab size: ", vocab_size)
token_ids = 2
print('first story decoded: ', decode(all_stories[token_ids][0].tolist(), language_id=all_stories[token_ids][1][0]))
# PADDING_TOKEN_IDX= encode(" ", 0)[0]


Hola encoded in english
[44, 1798]
Hola encoded in spani.tqdmsh
[580]
 didn
vocab size:  8192
first story decoded:  One day, a little boy was playing. He loved to play! Suddenly, knock, knock, knock! Someone knocked on the door. "Hello!" said the boy. "You are a visitor," said the boy. The visitor said, "Hello! I have a gift for you." The boy was very happy. "Thank you!" he said.

The visitor gave the boy a big gift. It was a toy truck! The truck was really cool. It had big wheels and a horn that honked. The boy jumped into the truck and started to play. It was the best day of his life!

The boy played and played with the truck. He took it everywhere. He loved the truck. Suddenly, the truck started making a strange noise. Crack, crack, crack! The boy got scared. The truck broke. How sad!

The boy started to cry. It really hurt him that his truck was broken! The visitor saw him and said, "Don’t worry!" The visitor waved his hand. Suddenly, the truck was new again! The truck was fixed!



In [118]:
all_stories[0][0]

tensor([   0,    3,  552,  273,  265, 1008,  403,   18,  272,  636,  273,  963,
          18,  407,  310,  273,  490,    5,  312,  577, 1008, 3005,   18,  203,
         203,  268,  310,  367,  265,  326,   18,  272,  326,  273,  354,   18,
         272,  326,  356,  265, 1212,  969,    5,  272,  310,  297,   16,  285,
         570,   16,  326,  311,  203,  203,  268,  326, 1023,   18,  272,  326,
         273, 1149,    5,  272,  310,  603,  267,  326,  265,  648,   18,  272,
         326,  577,  267,  648,   18,  203,  203,  268,  310,  392,  494,   18,
         272,  310,  395,  357,  658,  265,  578,   18,  272,  578,  273,  963,
           5,  272,  310,  619,  267,  578,   18,  312,  577,  267,  578,    5,
         203,  203,  552,  273, 4317,   18,  272,  310,  273,  351,    5,  272,
         310,  356,  538,  545,   18,  437,  400,  587,  765,  276,  319,  640,
           5,    1], device='cuda:0', dtype=torch.int16)

In [119]:
n = int(0.9*len(all_stories))

train_data = all_stories[:n]
val_data = all_stories[n:]

In [120]:
train_data[7]


[tensor([   0,    3,  442,   16,  384, 1594,  349,  265,  645,   18,  272,  645,
          273,  354,   16,  339,  265,  354,  831,  303,  888, 2843,   18,  781,
          712,  276,  319,  349,  267,  645,    5,  203,  203,  486,  403,   16,
          267, 1082,  273, 3655,  337, 1110,   18,  272, 1082,  658,  265,  559,
          419,  903,    5,  272,  645, 2564,   16,  303,  267,  831,  811,   18,
          203,  203,  268,  831,  811,  303,  267,  645,  755,  607,    5,  272,
          645,  755,  349,  267, 1082,   18,  272,  645,  755,  337,  802,    5,
          203,  203,  552,  273, 1086,    5,  272,  645, 1063,  303, 1063,   16,
          303,  846, 2578,  360, 1283,    5,  203,  203,  268,  645,  273,  349,
          265,  629, 1052,   18,  407, 1008,  303,  982, 1052,    5,  333,  273,
          265,  629, 1052,  276, 2309,    5,    1], device='cuda:0',
        dtype=torch.int16),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 

In [121]:
t = 3

a=[1,2,3,4,5,6,7,8,9]

a[:9]

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [122]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data), (B,)) 
    # ix = [i for i in range(B)]
    
    x = torch.full((B, T), PAD_TOK, dtype=torch.long)
    y = torch.full((B, T), PAD_TOK, dtype=torch.long)
    x_lang_idxs = torch.full((B, T), 0, dtype=torch.long)
    y_lang_idxs = torch.full((B, T), 0, dtype=torch.long)

    for sequence_index, random_story_index in enumerate(ix):
        story_tokens = data[random_story_index][0].long()[:T - 1]
        story_length = story_tokens.shape[0]
        story_lang_idxs = data[random_story_index][1].long()[:T - 1]
        # print(story_lang_idxs)
        x[sequence_index][0:story_length-1] = story_tokens[0:story_length-1]
        y[sequence_index][0:story_length-1] = story_tokens[1:story_length]

        last_token_lang_idx =  story_lang_idxs[-1]
        x_lang_idxs[sequence_index] = torch.full_like(x_lang_idxs[sequence_index], last_token_lang_idx)
        y_lang_idxs[sequence_index] = torch.full_like(y_lang_idxs[sequence_index], last_token_lang_idx)
        x_lang_idxs[sequence_index][0: story_lang_idxs.shape[0]-1] = story_lang_idxs[1:story_lang_idxs.shape[0]]
        y_lang_idxs[sequence_index][0: story_lang_idxs.shape[0]-1] = story_lang_idxs[0:story_lang_idxs.shape[0]-1]


    return x, y, x_lang_idxs, y_lang_idxs

xb, yb, x_lang_idxs, y_lang_idxs = get_batch('train')

# print(xb[0])
# print(yb[0])

print(xb[0][:5])
print(yb[0][:5])
print(x_lang_idxs[0][:5])
print(y_lang_idxs[0][:5])
language = x_lang_idxs[0,0]
print(language)
print(decode(xb[0].tolist(), language.item())) # the zero on xb is the first story. 


y_lang_idxs


tensor([   0,    3, 4639,  203,   37], device='cuda:0')
tensor([   3, 4639,  203,   37,  390], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0], device='cuda:0')
tensor(0, device='cuda:0')
  
A little dog barks. Woof! Woof! He sees rocks. They are big! He wants to play. He goes to the rocks!

The dog looks for another dog. He barks! But there is no other dog! He is alone! The dog feels sad.

He sees something. It’s a cat! The cat is on the rocks! The dog is happy. He has a friend!

The cat looks at the dog. The cat doesn’t want to play. He wants to sleep! The dog feels sad. He is alone again!

The dog leaves. He goes to find another friend. He wants to play! He doesn’t want to be alone!  



tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')

In [123]:
class Head(nn.Module):
    '''One Head of self-attention'''
    def __init__(self, H):
        super().__init__()
        self.query = nn.Linear(C, H, bias=False)
        self.key = nn.Linear(C, H, bias=False)
        self.value = nn.Linear(C, H, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(T, T)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        query_vectors = self.query(x)
        key_vectors = self.key(x)


        # Attention masking(so we can't look into the past):
        tril = self.tril
        wei = torch.zeros(T, T) 
        wei = wei.masked_fill(tril == 0, float('-inf')) # set the upper triangular to -inf

        # multiply the two to get the attention weights
        attention_pattern = query_vectors @ key_vectors.transpose(-2, -1) # T, T
        attention_pattern = attention_pattern / (H ** 0.5) # scale the attention pattern for numerical stability
        attention_weights = F.softmax(attention_pattern + wei, dim=-1) # T, T (the row dimension is the query)
        attention_weights = self.dropout(attention_weights)

        value_vectors = self.value(x) # the direction we should go in the embedding space for each token (ie more blue) T, H

        # apply the attention weights to the value vectors
        context = attention_weights @ value_vectors # T, H

        # project back into original space from value space
        return context

x = torch.randn(B,T,C)
head = Head(H)

In [124]:
class MultiHeadAttention(nn.Module):
    '''Multiple heads of self-attention'''
    def __init__(self, H, C, n_heads): # H is head embedding space size, n_heads is number of heads
        super().__init__()
        self.heads = nn.ModuleList([Head(H) for _ in range(n_heads)])
        self.combine_heads = nn.Linear(H*n_heads, C)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        x = torch.cat([head(x) for head in self.heads], dim=-1)
        x = self.combine_heads(x)  # T, C
        return self.dropout(x)

In [125]:
head = MultiHeadAttention(H, C, n_heads)
head.heads[0].forward(x).shape

torch.Size([64, 256, 32])

In [126]:
class FeedForward(nn.Module):
    '''Feed-forward neural network'''
    def __init__(self, C):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(C, C * feedforward_factor),
            nn.ReLU(),
            nn.Linear(C * feedforward_factor, C),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [127]:
class LayerNorm(nn.Module):
    '''Layer normalization'''
    def __init__(self, C, use_affine=True):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(C)) if use_affine else None
        self.beta = nn.Parameter(torch.zeros(C)) if use_affine else None
        
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        if self.gamma is not None and self.beta is not None:
            return self.gamma * (x - mean) / (std + 1e-6) + self.beta
        else:
            return (x - mean) / (std + 1e-6)

In [128]:
class Block(nn.Module):
    '''Transformer block'''
    def __init__(self, H, C, n_heads):
        super().__init__()
        self.attention = MultiHeadAttention(H, C, n_heads)
        self.ff = FeedForward(C)
        self.norm1 = LayerNorm(C, use_affine=True)
        self.norm2 = LayerNorm(C, use_affine=True)

    def forward(self, x):
        x = x + self.attention(self.norm1(x))
        x = x + self.ff(self.norm2(x))
        return x

In [136]:
class MultiTokenizerGPT(nn.Module):
    def __init__(self, n_layers, n_languages):
        super().__init__()
        self.token_embedding_tables = [nn.Embedding(vocab_size, C) for _ in range(n_languages)]
        self.position_embedding_table = nn.Embedding(T, C)
        self.lm_heads = nn.ModuleList([nn.Linear(C, vocab_size) for _ in range(n_languages)])
        self.layers = nn.ModuleList([Block(H, C, n_heads) for _ in range(n_layers)])
        self.block = nn.ModuleList([Block(H, C, n_heads)])
    
    # token_lang_idx has same shape as token_ids, encodes langs
    def forward(self, token_ids, token_lang_idxs, targets=None):
        B, T = token_ids.shape

        pos_emb = self.position_embedding_table(torch.arange(T))

        token_emb = torch.zeros(B*T, C)

        token_ids_flat = token_ids.view(-1)

        unique_lang_idxs = torch.unique(token_lang_idxs)

        # List of tensors, where each tensor containds the indexes in the flat tokens list of all tokens of a language.
        # there are as many tensors as there are languages present in the batch. Total length of all tensors = # tokens
        lang_tok_mapping = [torch.where(token_lang_idxs.view(-1) == idx) for idx in unique_lang_idxs]


        for lang_id, monolingual_tokens_idx in zip(unique_lang_idxs, lang_tok_mapping):
            lang_embedding_table = self.token_embedding_tables[lang_id]
            monolingual_tok_emb = lang_embedding_table(token_ids_flat[monolingual_tokens_idx])
            token_emb[monolingual_tokens_idx] = monolingual_tok_emb
        


        x = token_emb.view(B, T, C) + pos_emb # token identities and positions contained

        for layer in self.layers:
            x = layer(x)


        x_flat = x.view(B*T, C)
        logits = torch.zeros(B*T, vocab_size)

        for lang_id, monolingual_tokens_idx in zip(unique_lang_idxs, lang_tok_mapping):
            lm_head = self.lm_heads[lang_id]
            new_logits = lm_head(x_flat[monolingual_tokens_idx])
            logits[monolingual_tokens_idx] = new_logits

        logits = logits.view(B,T,vocab_size)


        batch_dim, sequence_dim, embedding_dim = logits.size()

        if targets is None:
            return logits, None
        else:
            logits_loss_view = logits.view(-1, vocab_size) 
            targets_loss_view = targets.view(-1)
            loss = F.cross_entropy(logits_loss_view, targets_loss_view)
            return logits, loss

        return idx
    def prompt_model(self, prompt, max_new_tokens, language="spanish", temperature=0.5):
        language_token = None
        language_id = 0
        if language == "english":
            language_token = BEGIN_EN_TOK
            language_id = 0
        elif language == "spanish":
            language_token = BEGIN_ES_TOK
            language_id = 1
        else:
            raise ValueError(f"invalid language {language}")
        
        token_lang_idxs = torch.full([1, T], language_id)

        autoregressive_seq = [START_TOK, language_token]

        autoregressive_seq += encode(prompt, language_id)

        for _ in range(max_new_tokens):
            prediction_index = len(autoregressive_seq)-1

            model_input = torch.tensor(autoregressive_seq)
            
            while model_input.shape[0] < T:
                pad_token = torch.tensor([PAD_TOK], dtype=torch.long)
                model_input = torch.cat((model_input, pad_token), dim=0)

            model_input = model_input.unsqueeze(0)

            logits, loss = model.forward(
                token_ids=model_input,
                token_lang_idxs=token_lang_idxs
                )
            prediction_token = logits[:, prediction_index, :] / temperature
            probabilities = F.softmax(prediction_token, dim=-1)
            next_token = torch.multinomial(probabilities, num_samples=1)
            next_token = next_token.item()
            if(next_token == END_TOK):
                break

            autoregressive_seq.append(next_token)
        # get the autoregressive sequence
        return decode(autoregressive_seq, language_id)


model = MultiTokenizerGPT(n_layers, 2)

In [137]:
xb, yb, x_lang_idxs, y_lang_idxs = get_batch('train')

logits, loss = model(
    token_ids = xb,
    token_lang_idxs = x_lang_idxs,
    targets = yb
)
print(logits.shape)
print(loss)

# test_idx = torch.zeros(1, T).long()
# model.forward(idx=test_idx)
# decode(model.generate(idx=test_idx, max_new_tokens=100)[0].tolist())

torch.Size([64, 256, 8192])
tensor(9.2287, device='cuda:0', grad_fn=<NllLossBackward0>)


In [138]:
# get the number of parameters in the model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("number of parameters in the model (millions): ", count_parameters(model) /1e6)

number of parameters in the model (millions):  12.825856


In [139]:
token_ids = torch.zeros(1, 1).long()
token_ids[:,-T:]

tensor([[0]], device='cuda:0')

In [140]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [141]:
eval_iters = 10
eval_interval = 300

@torch.no_grad()
def estimate_loss(is_last=False):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        real_iters = eval_iters
        if is_last and split == 'val':  # increase last eval to mitigate noise
            real_iters *= 10 
        losses = torch.zeros(real_iters)
        for k in range(real_iters):
            # X, Y = get_batch(split)
            xb, yb, x_lang_idxs, y_lang_idxs = get_batch(split)

            logits, loss = model(
                token_ids = xb,
                token_lang_idxs = x_lang_idxs,
                targets = yb
            )
            losses[k] = loss.item()
        out[split] = losses.mean() / chars_per_token
    model.train()
    return out
estimate_loss()

{'train': tensor(2.3641, device='cuda:0'),
 'val': tensor(2.3611, device='cuda:0')}

In [142]:
dump_model_interval = 1000
chars_per_token=3.9

#print('loading last model')
#model.load_state_dict(torch.load('tiny-stories-model-12.pt'))


for steps in tqdm(range(0, max_iters)):
    xb, yb, x_lang_idxs, y_lang_idxs = get_batch('train')
    # loss
    logits, loss = model(
        token_ids = xb,
        token_lang_idxs = x_lang_idxs,
        targets = yb
    )
    optimizer.zero_grad(set_to_none=True)

    loss.backward()
    optimizer.step()
    if steps % eval_interval == 0:
        losses = estimate_loss()
        # wandb.log({"train": losses['train'].item(), "val": losses['val'].item(), "l2":l2})
        print({"train": losses['train'].item(), "val": losses['val'].item()})
    if steps % dump_model_interval == 0 and steps > 0:
        model_no = steps // dump_model_interval
        torch.save(model.state_dict(), f'tiny-stories-model-{model_no}.pt')

losses = estimate_loss(is_last=True)

  0%|          | 2/60000 [00:01<7:41:08,  2.17it/s] 

{'train': 2.1513853073120117, 'val': 2.1557843685150146}


  1%|          | 302/60000 [00:49<4:41:38,  3.53it/s]

{'train': 0.5170854330062866, 'val': 0.5216456651687622}


  1%|          | 602/60000 [01:35<5:16:23,  3.13it/s]

...

{'train': 0.2553711235523224, 'val': 0.27574536204338074}


 19%|█▊        | 11102/60000 [28:30<4:39:55,  2.91it/s]

{'train': 0.24981610476970673, 'val': 0.27189043164253235}


 19%|█▊        | 11127/60000 [28:33<2:05:27,  6.49it/s]


KeyboardInterrupt: 

In [ ]:
estimate_loss()

In [ ]:
torch.save(model.state_dict(), 'tiny-stories-multilingual-model.pt')

In [ ]:
# model.load_state_dict(torch.load('tiny-stories-model-10.pt'))

<All keys matched successfully>

In [154]:
# test_idx = torch.zeros(1, T).long() * 198
# print(decode(
#     model.generate(idx=test_idx, max_new_tokens=C)[0].tolist()
# )[T:])
model.prompt_model(
        prompt="Diego tenia un perro grande.",
    max_new_tokens=100,
    language="english",
    temperature=0.7
)

'Diego tenia un perro grande. He jumps high in a tall tree. He likes to jump!\n\nOne day, Diego sees an insect. It is very cute. "Hello, insect," says Diego. The insect flies away. Diego is sad.\n\n"Don\'t be sad," says Mama Bird. "The insect won\'t talk."\nDiego looks at the cat. The cat jumps and sings. "That is not a good insect," says the cat. "But it is a pretty butterfly!"\n\nDiego'

# Multitokenizer Toy Implemetation